In [3]:
import os
from dotenv import load_dotenv
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.document_loaders import GoogleDriveLoader

load_dotenv(override=True)

embeddings = OpenAIEmbeddings()

index_name = os.getenv("PINECONE_INDEX_NAME")

ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [1]:


loader = GoogleDriveLoader(document_ids=["1BT5apJMTUvG9_59-ceHbuZXVTJKeyyknQsz9ZNIEwQ8"],
                          credentials_path="../../desktop_credetnaisl.json")
docs = loader.load()
print(docs[0].text)

ModuleNotFoundError: Module langchain_community.document_loaders not found. Please install langchain-community to access this module. You can install it using `pip install -U langchain-community`